<a href="https://colab.research.google.com/github/RishiSoni08/Capacitated_facility_location_Problem/blob/master/Capacitated_Facility_location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facility Location Problem

In [ ]:
import pandas as pd
import numpy as np
import folium
from geopy.distance import geodesic
import json

# new



In [ ]:
# loading data
with open('combined_data.json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
facility_data = data["facility_data"]
demand_data = data["demand_data"]

In [ ]:
map = folium.Map(location=(23.25,77.41),zoom_start =5,max_zoom=10,min_zoom=5)



import

In [ ]:
faci_cities = list(facility_data.keys())
dema_cities = list(demand_data.keys())
num_faci = len(faci_cities)
num_dema=len(dema_cities)



In [ ]:
val = geodesic((19,72),(28,77)).kilometers
print(type(val))

<class 'float'>


In [ ]:
# initializing the matrices
columns = ['city','capacity','cost']

faci = pd.DataFrame(facility_data).transpose() ## capacity of  city.
dema = pd.DataFrame(demand_data).transpose() ## demand  each city.

distance_matrix = np.zeros((num_faci,num_dema))

total_demand= dema['demand'].sum()

In [ ]:
#plotting demand location
for i in range (50):
    cord = (dema.iloc[i][0],dema.iloc[i][1])
    folium.Marker(location=cord,
                  icon= folium.Icon(icon='glyphicon-map-marker',
                                     prefix='glyphicon',
                                     icon_size=(5,5),
                                     icon_color='red',shadow_size=(0,0)),
                    popup=dema.index[i]).add_to(map)
#plotting Available facilities

for i in range (20):
    cord = (faci.iloc[i][0],faci.iloc[i][1])
    folium.Marker(location=cord,
                  icon= folium.Icon(icon='glyphicon-map-marker',
                                     prefix='glyphicon',
                                     icon_size=(5,5),
                                     icon_color='blue',shadow_size=(0,0)),
                    popup=faci.index[i]).add_to(map)

map

/tmp/ipykernel_24580/1438127255.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cord = (dema.iloc[i][0],dema.iloc[i][1])
/tmp/ipykernel_24580/1438127255.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cord = (faci.iloc[i][0],faci.iloc[i][1])


In [ ]:
for i, city1 in enumerate(faci_cities):
    for j, city2 in enumerate(dema_cities):
        if i==j :
            dist = 0
        else:
            cords1 = (facility_data[city1]['latitude'],facility_data[city1]['longitude'])
            cords2 = (demand_data[city2]['latitude'],demand_data[city2]['longitude'])
            dist = geodesic(cords1,cords2).kilometers
            distance_matrix[i][j]= dist

## Objective Function

In [ ]:
# objective is to minimize the cost of the setting up the facility and the cost of
#transportation (in this case we will minimize the distance)

import pulp

In [ ]:
problem = pulp.LpProblem('Location_allocation_problem', pulp.LpMinimize)


In [ ]:
distance_matrix.shape

(20, 50)

In [ ]:
# Yi = 1 if facility i is chosen
y = pulp.LpVariable.dicts('Y',range(num_faci),cat=pulp.LpBinary)
x = pulp.LpVariable.dicts('X',(range(num_faci),range(num_dema)),cat=pulp.LpBinary)

In [ ]:
### sum(Ci Yi) + sum(xij* Dij)
# Ci = cost, Dij is distance
problem += pulp.lpSum(
    y[i] * faci.iloc[i, 3] for i in range(num_faci)
) + pulp.lpSum(
    x[i][j] * distance_matrix[i][j]
      for i in range(num_faci) for j in range(num_dema)
)

In [ ]:
#  sum j to n  (xij* pj) <= Qi yi for every i
# ; pj is demand of node j and q i is the capacity of node i
# sum_of_capacities is more than total demand.
# adding constraints
for i in range(num_faci):
    problem += pulp.lpSum(((dema.iloc[j,2]*x[i][j]))
                        for j in range (num_dema))<= (faci.iloc[i,2]*y[i]), f'for facility_{i}'

In [ ]:
# sum i to n xij = 1 , for every j
# every facility is connected to one demand point only

for j in range(num_dema):
    problem += pulp.lpSum(x[i][j] for i in range(num_faci))==1, f'facility{i} is connected to {j}'

In [ ]:
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/delhivery/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d3ea7c4cc8ff4ee58cfc569a968478e2-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/d3ea7c4cc8ff4ee58cfc569a968478e2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 75 COLUMNS
At line 5125 RHS
At line 5196 BOUNDS
At line 6217 ENDATA
Problem MODEL has 70 rows, 1020 columns and 2020 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 14187 - 0.00 seconds
Cgl0004I processed model has 70 rows, 1020 columns (1020 integer (1020 of which binary)) and 2020 elements
Cbc0038I Initial state - 20 integers unsatisfied sum - 5.0456
Cbc0038I Pass   1: suminf.    4.54560 (19) obj. 15287 iterations 4
Cbc0038I Solution found of 42180.7
Cbc0038I Before mini branch and bound, 1000 integers at bound fixed 

1

In [ ]:
# Print the status
print("Status:", pulp.LpStatus[problem.status])

Status: Optimal


In [ ]:
# printing the total facilities open
for j in range(num_faci):
    if pulp.value(y[j])==1:
        print(f'facility {faci_cities[j]} is open')

facility Mumbai is open
facility Delhi is open
facility Nagpur is open
facility Patna is open
facility Ludhiana is open
facility Coimbatore is open


In [ ]:
# for demand in range(num_dema):
#     for facility in range(num_faci):
#         if pulp.value(x[facility][demand])==1:
#             print(f'{faci.index[facility]} is supplying to {dema.index[demand]}')


In [ ]:
#printing the locations

# Plot the route
for demand in range(num_dema):
    for facility in range(num_faci):
        if pulp.value(x[facility][demand])==1:
            waypoints = [(faci.iloc[facility][0],faci.iloc[facility][1]),
                         (dema.iloc[demand][0],dema.iloc[demand][1])]
            #print(f'{data.index[facility]} is supplying to {data.index[demand]}')
            folium.PolyLine(locations=waypoints, color='blue', weight=2, opacity=1).add_to(map)



/tmp/ipykernel_24580/899252110.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  waypoints = [(faci.iloc[facility][0],faci.iloc[facility][1]),
/tmp/ipykernel_24580/899252110.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  (dema.iloc[demand][0],dema.iloc[demand][1])]


In [ ]:
map

In [ ]:
sum_capa =0
for i in range(num_faci):

    if pulp.value(y[i])==1:
        sum_capa+= faci.iloc[i,2]
        print(f'{faci_cities[i] }, Capacity {faci.iloc[i,2]} : ')
    sum_dema= 0
    for j in range(num_dema):
        # Plot the route
        if pulp.value(x[i][j])==1:
            sum_dema+=dema.iloc[j,2]
            print(f'\t {dema_cities[j]}, Demand : {dema.iloc[j,2]}')

    if pulp.value(y[i])==1:
        print(f'Total Demand : {sum_dema}')

print(f'total capacity of open facilities is {sum_capa}')




Mumbai, Capacity 5000.0 : 
	 Mumbai, Demand : 500.0
	 Ahmedabad, Demand : 300.0
	 Pune, Demand : 400.0
	 Surat, Demand : 300.0
	 Nashik, Demand : 300.0
	 Rajkot, Demand : 250.0
	 Aurangabad, Demand : 250.0
	 Navi Mumbai, Demand : 300.0
	 Solapur, Demand : 200.0
	 Hubli, Demand : 250.0
Total Demand : 3050.0
Delhi, Capacity 4500.0 : 
	 Delhi, Demand : 600.0
	 Jaipur, Demand : 350.0
	 Kanpur, Demand : 400.0
	 Agra, Demand : 250.0
	 Faridabad, Demand : 250.0
	 Meerut, Demand : 200.0
	 Gwalior, Demand : 250.0
	 Jodhpur, Demand : 250.0
	 Kota, Demand : 200.0
	 Bareilly, Demand : 200.0
	 Aligarh, Demand : 250.0
	 Moradabad, Demand : 200.0
	 Gurgaon, Demand : 300.0
Total Demand : 3700.0
Nagpur, Capacity 2500.0 : 
	 Hyderabad, Demand : 400.0
	 Lucknow, Demand : 450.0
	 Nagpur, Demand : 350.0
	 Jabalpur, Demand : 200.0
	 Vijayawada, Demand : 300.0
	 Raipur, Demand : 250.0
Total Demand : 1950.0
Patna, Capacity 2500.0 : 
	 Kolkata, Demand : 500.0
	 Visakhapatnam, Demand : 400.0
	 Varanasi, Demand 